# Combining all together

Now we have all the pieces necessary to make implement the interactive optimization process using clustering as surrogates and different scalarization functions. Yeah.

In [1]:
 %matplotlib inline
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from ASF import ASF
from gradutil import *
from pyomo.opt import SolverFactory
seedn = 2

First lets take all the data in

In [2]:
%%time
revenue, carbon, deadwood, ha = init_boreal()
n_revenue = nan_to_bau(revenue)
n_carbon= nan_to_bau(carbon)
n_deadwood = nan_to_bau(deadwood)
n_ha = nan_to_bau(ha)
ide = ideal(False)
nad = nadir(False)
opt = SolverFactory('glpk')

In [3]:
x = pd.concat((n_revenue, n_carbon, n_deadwood, n_ha), axis=1)
x_stack = np.dstack((n_revenue, n_carbon, n_deadwood, n_ha))

Normalize all the columns in 0-1 scale

In [4]:
norm_revenue = new_normalize(n_revenue)
norm_carbon = new_normalize(n_carbon)
norm_deadwood = new_normalize(n_deadwood)
norm_ha = new_normalize(n_ha)

x_norm = np.concatenate((norm_revenue, norm_carbon, norm_deadwood, norm_ha), axis=1)
x_norm_stack = np.dstack((norm_revenue, norm_carbon, norm_deadwood, norm_ha))

Cluster the data to some clusters and calculate correponding weights

In [5]:
%%time 
nclust1 = 200
c, xtoc, dist = cluster(x_norm, nclust1, seedn, verbose=1)
nvar = len(x_norm)
w = np.array([sum(xtoc == i) for i in range(nclust1)])/nvar

Calculate new cluster centers using average from normalized data

In [6]:
c_mean = np.array([x_norm_stack[xtoc == i].mean(axis=0) for i in range(nclust1)])

Calculate new cluster centers using stand closest to the center of the cluster. This option could be more justified in the practical level, because we don't know so much about the forestry and features of forest stands.

In [7]:
from scipy.spatial.distance import euclidean
c_close = np.array([x_norm_stack[min(np.array(range(len(xtoc)))[xtoc == i],
                                     key=lambda index: euclidean(x_norm[index],
                                                                 np.mean(x_norm[xtoc == i], axis=0)))]
                    for i in range(nclust1)
                    if sum(xtoc == i) > 0])


Calculate solution for some reference using the cluster centers

In [8]:
ref = np.array((ide[0], 0, 0, 0))
mean_asf = ASF(ide, nad, ref, c_mean, weights=w, nvar=nvar)
res_asf_mean = opt.solve(mean_asf.model)
mean_stom = ASF(ide, nad, ref, c_mean, weights=w, nvar=nvar, scalarization='stom')
res_stom_mean = opt.solve(mean_stom.model)
mean_guess = ASF(ide, nad, ref, c_mean, weights=w, nvar=nvar, scalarization='guess')
res_stom_mean = opt.solve(mean_guess.model)

In [28]:
(model_to_real_values(x_stack, mean_asf.model, xtoc),
model_to_real_values(x_stack, mean_stom.model, xtoc),
model_to_real_values(x_stack, mean_guess.model, xtoc),
ide)

Calculate solution for some reference using the stands closest to the cluster center

In [20]:
ref = np.array((ide[0], 0, 0, 0))
close_asf = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar)
res_close = opt.solve(close_asf.model)
close_stom = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar, scalarization='stom')
res_stom_close = opt.solve(close_stom.model)
close_guess = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar, scalarization='guess')
res_stom_close = opt.solve(close_guess.model)

In [21]:
(model_to_real_values(x_stack, close_asf.model, xtoc),
model_to_real_values(x_stack, close_stom.model, xtoc),
model_to_real_values(x_stack, close_guess.model, xtoc),
ide)

In [22]:
ref = np.array((0, ide[1], 0, 0))
close_asf = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar)
res_close = opt.solve(close_asf.model)
close_stom = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar, scalarization='stom')
res_stom_close = opt.solve(close_stom.model)
close_guess = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar, scalarization='guess')
res_stom_close = opt.solve(close_guess.model)

In [23]:
(model_to_real_values(x_stack, close_asf.model, xtoc),
model_to_real_values(x_stack, close_stom.model, xtoc),
model_to_real_values(x_stack, close_guess.model, xtoc),
ide)

In [24]:
ref = np.array((0, 0,ide[2], 0))
close_asf = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar)
res_close = opt.solve(close_asf.model)
close_stom = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar, scalarization='stom')
res_stom_close = opt.solve(close_stom.model)
close_guess = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar, scalarization='guess')
res_stom_close = opt.solve(close_guess.model)

In [25]:
(model_to_real_values(x_stack, close_asf.model, xtoc),
model_to_real_values(x_stack, close_stom.model, xtoc),
model_to_real_values(x_stack, close_guess.model, xtoc), 
ide)

In [26]:
ref = np.array((0, 0, 0, ide[3]))
close_asf = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar)
res_close = opt.solve(close_asf.model)
close_stom = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar, scalarization='stom')
res_stom_close = opt.solve(close_stom.model)
close_guess = ASF(ide, nad, ref, c_close, weights=w, nvar=nvar, scalarization='guess')
res_stom_close = opt.solve(close_guess.model)

In [27]:
(model_to_real_values(x_stack, close_asf.model, xtoc),
model_to_real_values(x_stack, close_stom.model, xtoc),
model_to_real_values(x_stack, close_guess.model, xtoc),
ide)

It really looks like that the scalarizations are emphasizing the objective we are setting as the target. It looks fine then.

Also can say that using the "close" values we get more averaged results, but anyway they are more justified than just the means of clusters.